<a href="https://colab.research.google.com/github/okada-t-rafael/nlp_analysis_study/blob/master/Assignment%2B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
from google.colab import files
files.upload()

Saving dates.txt to dates.txt


{'dates.txt': b'03/25/93 Total time of visit (in minutes):\n6/18/85 Primary Care Doctor:\nsshe plans to move as of 7/8/71 In-Home Services: None\n7 on 9/27/75 Audit C Score Current:\n2/6/96 sleep studyPain Treatment Pain Level (Numeric Scale): 7\n.Per 7/06/79 Movement D/O note:\n4, 5/18/78 Patient\'s thoughts about current substance abuse:\n10/24/89 CPT Code: 90801 - Psychiatric Diagnosis Interview\n3/7/86 SOS-10 Total Score:\n(4/10/71)Score-1Audit C Score Current:\n(5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC_12.6Activities of Daily Living (ADL) Bathing: Independent\n4/09/75 SOS-10 Total Score:\n8/01/98 Communication with referring physician?: Done\n1/26/72 Communication with referring physician?: Not Done\n5/24/1990 CPT Code: 90792: With medical services\n1/25/2011 CPT Code: 90792: With medical services\n4/12/82 Total time of visit (in minutes):\n1; 10/13/1976 Audit C Score, Highest/Date:\n4, 4/24/98 Relevant Drug History:\n) 59 yo unemployed w referred by Urgent Care for psychiatr

In [2]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [354]:
def date_sorter():
    
    # Your code here
    month_dict = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12,
    'January': 1, 'Janaury': 1,'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12, 'Decemeber': 12
    }

    df2 = df.str.extract(r'(?P<t>(?P<m>\d{1,2})[/-](?P<d>\d{1,2})[/-](?P<y>\d{2,4}))')
    df2.dropna(inplace=True)
    df2['id'] = df2.index
    df2['m'] = df2['m'].astype(int)
    df2['d'] = df2['d'].astype(int)
    df2['y'] = df2['y'].apply(lambda y: int(y) + 1900 if len(y) == 2 else int(y))

    df3 = df.str.extract(r'(?P<t>(?P<d>\d{1,2}) (?P<m>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|June|July|August|September|October|November|December) (?P<y>\d{4}))')
    df3.dropna(inplace=True)
    df3['id'] = df3.index
    df3['m'] = df3['m'].apply(lambda m: month_dict[m])
    df3['d'] = df3['d'].astype(int)
    df3['y'] = df3['y'].astype(int)

    df4 = df.str.extract(r'(?P<t>(?P<m>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|June|July|August|September|October|November|December)[- .]+(?P<d>\d{1,2})[- ,]+(?P<y>\d{4}))')
    df4.dropna(inplace=True)
    df4['id'] = df4.index
    df4['m'] = df4['m'].apply(lambda m: month_dict[m])
    df4['d'] = df4['d'].astype(int)
    df4['y'] = df4['y'].astype(int)

    df5 = df.str.extract(r'(?P<t>(?<!\d).{2}(?P<m>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|Janaury|February|March|April|June|July|August|September|October|November|December|Decemeber)[ ,.]+(?P<y>\d{4}))')
    df5.dropna(inplace=True)
    df5['id'] = df5.index
    df5['m'] = df5['m'].apply(lambda m: month_dict[m])
    df5['d'] = 1
    df5['y'] = df5['y'].astype(int)

    df6 = df.str.extract(r'(?P<t>^[^\s]?(?P<m>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|June|July|August|September|October|November|December)[ ,.]+(?P<y>\d{4}))')
    df6.dropna(inplace=True)
    df6['id'] = df6.index
    df6['m'] = df6['m'].apply(lambda m: month_dict[m])
    df6['d'] = 1
    df6['y'] = df6['y'].astype(int)

    df7 = df.str.extract(r'(?P<t>^(?P<m>\d{1,2})/(?P<y>\d{4})[^/])')
    df7.dropna(inplace=True)
    df7['id'] = df7.index
    df7['m'] = df7['m'].astype(int)
    df7['d'] = 1
    df7['y'] = df7['y'].astype(int)

    df8 = df.str.extract(r'(?P<t>[A-Za-z ~(-](?P<m>\d{1,2})/(?P<y>\d{4})[^/])')
    df8.dropna(inplace=True)
    df8['id'] = df8.index
    df8['m'] = df8['m'].astype(int)
    df8['d'] = 1
    df8['y'] = df8['y'].astype(int)

    df9 = df.str.extract(r'(?P<t>[^\s\d/-](?P<y>\d{4})[ ,.]*)')
    df9.dropna(inplace=True)
    df9['id'] = df9.index
    df9['m'] = 1
    df9['d'] = 1
    df9['y'] = df9['y'].astype(int)

    df10 = df.str.extract(r'(?P<t>(?!Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|ary|ury|rch|ril|une|uly|ust|ber|\d{2},| \d{2}).{3}\b (?P<y>\d{4})[ ,.]*)')
    df10.dropna(inplace=True)
    df10['id'] = df10.index
    df10['m'] = 1
    df10['d'] = 1
    df10['y'] = df10['y'].astype(int)

    df11 = df.str.extract(r'(?P<t>^(?P<y>\d{4})[ ,.]*)')
    df11.dropna(inplace=True)
    df11['id'] = df11.index
    df11['m'] = 1
    df11['d'] = 1
    df11['y'] = df11['y'].astype(int)

    df12 = df.str.extract(r'(?P<t>(?<=.Age \d{2}|Age, \d{2}), (?P<y>\d{4})[ ,.]*)')
    df12.dropna(inplace=True)
    df12['id'] = df12.index
    df12['m'] = 1
    df12['d'] = 1
    df12['y'] = df12['y'].astype(int)

    df13 = df.str.extract(r'(?P<t>[;:\-] (?P<y>\d{4})[^\d])')
    df13.dropna(inplace=True)
    df13['id'] = df13.index
    df13['m'] = 1
    df13['d'] = 1
    df13['y'] = df13['y'].astype(int)

    df_final = pd.concat([df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13])
    df_final.sort_values(by=['y', 'm', 'd'], inplace=True)
    df_final.reset_index(inplace=True)
    df_final['rank'] = df_final.index
    df_final.index = df_final['id']
    df_final.sort_index(inplace=True)
    df_final.index.name = None
    
    return df_final['rank']

In [355]:
date_sorter()

0      272
1      170
2        2
3       55
4      312
      ... 
495     96
496    408
497    430
498    402
499    106
Name: rank, Length: 500, dtype: int64